In [41]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import pandas as pd



#raw data

In [ ]:



df = pd.read_csv("dataset/interactions.csv")
print(df.to_string(index=False))


user_id item_id  score
  user8 item287    0.5
 user86 item262    0.8
 user59 item255    0.8
 user51 item280    1.0
 user37 item221    0.5
 user16 item253    0.5
  user3 item219    1.0
 user94  item42    0.8
 user23 item152    0.8
  user7 item241    0.8
 user43 item201    1.0
 user55 item165    0.8
user100 item210    0.5
  user8 item243    1.0
 user86 item232    1.8
 user93 item207    0.5
 user78 item205    1.0
 user19  item22    0.5
 user88  item82    0.4
 user84 item147    1.0
 user92 item243    0.5
  user9  item75    0.5
  user1  item28    0.8
 user36 item135    0.4
 user18 item139    0.6
 user25  item80    1.0
 user36 item197    0.8
 user16  item44    0.8
 user60 item149    0.5
 user65 item241    0.8
 user66  item33    0.5
 user90 item284    1.3
 user44 item184    0.8
 user66 item169    1.0
 user54 item284    1.0
 user75 item231    0.8
 user70   item1    0.5
  user2 item293    0.5
 user44 item197    0.5
 user51   item6    0.8
 user30  item94    0.7
 user64 item185    0.8
 user78 ite

In [34]:
df.shape  

(1500, 3)

In [33]:

df.describe()


,score
count,1500.000000
mean,0.769600
std,0.304024
min,0.200000
25%,0.500000
50%,0.800000
75%,1.000000
max,2.000000


#Normalize & Aggregate Interaction Scores


     handling:

Deduplication by summing interaction scores per (user_id, item_id)

Score normalization to the range [0.2, 5]

In [39]:



aggregated_df = df.groupby(['user_id', 'item_id'])['score'].sum().reset_index()

scaler = MinMaxScaler(feature_range=(0.2, 5))
aggregated_df['normalized_score'] = scaler.fit_transform(aggregated_df[['score']])


aggregated_df[['user_id', 'item_id', 'normalized_score']].head(10)


,user_id,item_id,normalized_score
0,user1,item104,1.266667
1,user1,item107,0.733333
2,user1,item111,1.800000
3,user1,item120,2.333333
4,user1,item150,1.000000
5,user1,item196,1.800000
6,user1,item221,1.800000
7,user1,item255,2.333333
8,user1,item266,2.333333
9,user1,item28,1.800000


**Resulting DataFrame with Normalized Scores**

**Train the SVD Model**

In [42]:


df = aggregated_df  
reader = Reader(rating_scale=(0, 5))


data = Dataset.load_from_df(df[['user_id', 'item_id', 'score']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


algo = SVD(n_factors=20, n_epochs=20, lr_all=0.005, reg_all=0.1, random_state=42)
algo.fit(trainset)


predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
print(f"Test RMSE: {rmse}")


RMSE: 0.3143
Test RMSE: 0.31426453580065955


**Define Function to Generate Recommendations**

In [43]:
def get_popular_items(interactions_data, top_n=10):
    pop_df = pd.DataFrame(interactions_data)
    return (
        pop_df.groupby("item_id")["score"]
        .sum()
        .sort_values(ascending=False)
        .head(top_n)
        .index
        .tolist()
    )

def generate_recommendations(algo, trainset, user_id, all_items, top_n=10, interactions_data=None):
    if user_id in trainset._raw2inner_id_users:
        user_items = set(trainset.to_raw_iid(iid) for (iid, _) in trainset.ur[trainset.to_inner_uid(user_id)])
        predictions = []

        for item_id in all_items:
            if item_id not in user_items:
                pred = algo.predict(user_id, item_id)
                predictions.append((item_id, pred.est))

        predictions.sort(key=lambda x: x[1], reverse=True)
        top_predictions = predictions[:top_n]
        recommended_items = [{"item_id": item_id, "score": round(score, 3)} for item_id, score in top_predictions]
    else:
        print(f"User {user_id} not in training data. Using popular item fallback.")
        popular_items = get_popular_items(interactions_data, top_n)
        recommended_items = [{"item_id": item_id, "score": None} for item_id in popular_items]

    return recommended_items


**Generate & Display Recommendations for a User**

In [46]:

interactions_data = df.to_dict("records")
all_items = set(df['item_id'].unique())


user_id = df['user_id'].iloc[0]  


recommendations = generate_recommendations(
    algo=algo,
    trainset=trainset,
    user_id=user_id,
    all_items=all_items,
    top_n=10,
    interactions_data=interactions_data
)


recommendations_df = pd.DataFrame(recommendations)


print(f"\nTop 10 recommended items for user '{user_id}':")
display(recommendations_df)



Top 10 recommended items for user 'user1':


,item_id,score
0,item91,0.938
1,item259,0.907
2,item58,0.887
3,item185,0.881
4,item46,0.877
5,item227,0.876
6,item166,0.868
7,item156,0.863
8,item257,0.861
9,item115,0.860


#item item  content based similarity